# CO2-concentration vs. air-temperature sonification

Using 1750-present global data.

In [ ]:
import ipytone
import numpy as np
import pandas as pd
import bqplot
import bqplot.pyplot as plt

In [ ]:
t = ipytone.transport

## Data

### Global CO2 concentration data

source: https://scrippsco2.ucsd.edu/data/atmospheric_co2/icecore_merged_products.html

In [ ]:
scripp_url = "https://scrippsco2.ucsd.edu/assets/data/"
co2_dataset = "atmospheric/merged_ice_core_mlo_spo/merged_ice_core_yearly.csv"

df_co2 = (
    pd.read_csv(
        scripp_url + co2_dataset,
        comment='"',
        header=None,
        names=["date", "co2_ppm"],
    )
    .assign(date=lambda df: df.date.astype(np.int64))
    .drop_duplicates("date")
    .set_index("date")
    .reindex(np.arange(1760, 2021))
    .interpolate()
    .dropna()
)

### Global temperature data

source: http://berkeleyearth.lbl.gov

In [ ]:
bkearth_url = "http://berkeleyearth.lbl.gov/auto/Global/"
temp_dataset = "Complete_TAVG_summary.txt"

df_temp = (
    pd.read_csv(
        bkearth_url + temp_dataset,
        comment="%",
        delim_whitespace=True,
        header=None,
        usecols=[0, 1],
        names=["date", "temp_anomaly"],
        index_col="date",
    )
    .assign(temp=lambda df: df.temp_anomaly + 8.6)
)

### Sound synthesis data pre-processing

In [ ]:
# duration of the audio clip
duration = 40.0

In [ ]:
df = df_temp.join(df_co2, how="inner")

In [ ]:
ppm = df.co2_ppm
df["co2_freq"] = (ppm - ppm.min()) / (ppm.max() - ppm.min()) * 1e3 + 200

In [ ]:
temp = df.temp
df["temp_freq"] = (temp - temp.min()) / (temp.max() - temp.min()) * 1e3 + 100

In [ ]:
rand01 = np.random.uniform(size=df.index.size)
df["temp_vel"] = np.where(rand01 > 0.5, 1, 0)

In [ ]:
date = df.index
df["play_time"] = (date - date[0]) / (date[-1] - date[0]) * duration

## Sound setup 

### CO2 synthesizer

In [ ]:
reverb = ipytone.Reverb(decay=7)
delay1 = ipytone.FeedbackDelay()
vibrato = ipytone.Vibrato()
co2_synth = ipytone.FMSynth().chain(
    vibrato, delay1, reverb, ipytone.destination
)

In [ ]:
co2_synth.oscillator.type = "sine2"
co2_synth.modulation.type = "square"
co2_synth.harmonicity.value = 2
co2_synth.modulation_index.value = 1.5
delay1.wet.value = 0.3
delay1.delay_time.value = 0.15
delay1.feedback.value = 0.5
reverb.wet.value = 0.4
vibrato.wet.value = 0.5
vibrato.frequency.value = 3.5
vibrato.depth.value = 0.25

In [ ]:
def play_synth(time):
    co2_synth.trigger_attack_release(df.co2_freq.iloc[0], duration, time=time)
    co2_synth.frequency.set_value_curve_at_time(
        list(df.co2_freq), time, duration
    )

co2_synth_eid = t.schedule(play_synth, 0)

### Temperature synthesizer

In [ ]:
delay = ipytone.PingPongDelay(
    delay_time=0.4, feedback=0.3, number_of_channels=2
)
vibrato2 = ipytone.Vibrato()
synth_temp = ipytone.PolySynth(volume=-12)
synth_temp.chain(delay, vibrato2, ipytone.destination)

In [ ]:
delay.delay_time.value = 0.13
delay.feedback.value = 0.7
delay.wet.value = 0.1

vibrato2.wet.value = 0.9
vibrato2.frequency.value = 2
vibrato2.depth.value = 1

synth_temp.voice.envelope.attack = 0.015
synth_temp.voice.envelope.decay = 0.01
synth_temp.voice.oscillator.type = "pulse"
synth_temp.voice.detune.value = -2000

In [ ]:
def clb(time, note):
    synth_temp.trigger_attack_release(
        note.note, 0.03, time=time, velocity=note.velocity
    )

In [ ]:
part = ipytone.Part(
    callback=clb,
    events=[
        {
            "note": r.temp_freq,
            "velocity": r.temp_vel,
            "time": r.play_time + np.random.uniform(-0.1, 0.1),
        }
        for _, r in df.iterrows()
    ]
)

In [ ]:
part.start()

## Figure

In [ ]:
sc_x = bqplot.LinearScale()
sc_y = bqplot.LinearScale()
scales = {"x": sc_x, "y": sc_y}

lines = bqplot.Lines(
    x=df.index, y=df.co2_ppm,
    scales=scales
)
lines.stroke_width = 4
lines.colors = ["red"]

vline_mark = plt.vline(df.index[0], scales=scales)
vline_mark.colors = ["black"]

ax_x = bqplot.Axis(scale=sc_x, label="Year")
ax_y = bqplot.Axis(
    scale=sc_y, orientation="vertical", label="CO2 concentration [ppm]"
)

fig = bqplot.Figure(marks=[lines, vline_mark], axes=[ax_x, ax_y])
fig.layout.width = "600px"

Show current position along the x-axis:

In [ ]:
date_range = df.index[-1] - df.index[0]

def move_vline(change):
    time = change["new"]
    date = time / duration * date_range + df.index[0]
    vline_mark.x = [date, date]


In [ ]:
t.schedule_observe(
    move_vline, update_interval=0.1, name="seconds", transport=True
)

Show figure and start sound:

In [ ]:
fig

In [ ]:
t.start().stop("+40")

## Clean-up

In [ ]:
t.clear(co2_synth_eid)
t.cancel(0)
t.schedule_unobserve(move_vline)

In [ ]:
part.dispose()
co2_synth.dispose()
synth_temp.dispose()
vibrato.dispose()
delay1.dispose()
reverb.dispose()
delay.dispose()
vibrato2.dispose()